In [1]:
pwd

'C:\\Users\\RADHA\\Documents\\gentask'

In [5]:
!pip install glob2

In [3]:
!pip install pypdf

     -------------------------------------- 254.8/254.8 kB 3.1 MB/s eta 0:00:00


In [10]:
!pip install python-docx
!pip install PyPDF2
!pip install --upgrade PyPDF2


In [11]:
import os
import glob
from PyPDF2 import PdfReader
from docx import Document

def pdf_to_docx(pdf_path, docx_path):
    pdf_reader = PdfReader(pdf_path)
    document = Document()

    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        document.add_paragraph(text)

    document.save(docx_path)

def convert_directory_pdf_to_docx(pdf_directory, docx_directory):
    # Create the output directory if it doesn't exist
    if not os.path.exists(docx_directory):
        os.makedirs(docx_directory)

    # Get a list of PDF files in the input directory
    pdf_files = glob.glob(os.path.join(pdf_directory, '*.pdf'))

    for pdf_file in pdf_files:
        # Construct the output file path
        base_name = os.path.basename(pdf_file)
        file_name = os.path.splitext(base_name)[0]
        docx_file = os.path.join(docx_directory, f'{file_name}.docx')

        # Convert the PDF file to DOCX
        pdf_to_docx(pdf_file, docx_file)

# Example usage
pdf_directory = r'C:\Users\RADHA\Documents\gentask\samplepdf'
docx_directory = r'C:\Users\RADHA\Documents\gentask\samplepdf2docx'

convert_directory_pdf_to_docx(pdf_directory, docx_directory)


In [12]:
import os
import glob
from PyPDF2 import PdfReader
from docx import Document

def pdf_to_docx(pdf_path, docx_path):
    pdf_reader = PdfReader(pdf_path)
    document = Document()

    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        document.add_paragraph(text)

    document.save(docx_path)

def convert_directory_pdf_to_docx(pdf_directory, docx_directory):
    # Create the output directory if it doesn't exist
    if not os.path.exists(docx_directory):
        os.makedirs(docx_directory)

    # Get a list of PDF files in the input directory
    pdf_files = glob.glob(os.path.join(pdf_directory, '*.pdf'))

    total_files = len(pdf_files)
    print(f"Converting {total_files} PDF files to DOCX...")

    for idx, pdf_file in enumerate(pdf_files, 1):
        # Construct the output file path
        base_name = os.path.basename(pdf_file)
        file_name = os.path.splitext(base_name)[0]
        docx_file = os.path.join(docx_directory, f'{file_name}.docx')

        # Convert the PDF file to DOCX
        pdf_to_docx(pdf_file, docx_file)

        # Print the progress
        print(f"Processed file {idx}/{total_files}")

    print("Conversion completed successfully!")

# Example usage
pdf_directory = r'C:\Users\RADHA\Documents\gentask\samplepdf'
docx_directory = r'C:\Users\RADHA\Documents\gentask\samplepdf2docx'

convert_directory_pdf_to_docx(pdf_directory, docx_directory)


Converting 1 PDF files to DOCX...
Processed file 1/1
Conversion completed successfully!


In [13]:
import os
import glob
from PyPDF2 import PdfReader
from docx import Document
import re

def sanitize_text(text):
    # Remove unsupported characters
    sanitized_text = re.sub(r'[^\x00-\x7F]', '', text)
    return sanitized_text

def pdf_to_docx(pdf_path, docx_path):
    pdf_reader = PdfReader(pdf_path)
    document = Document()

    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        sanitized_text = sanitize_text(text)
        document.add_paragraph(sanitized_text)

    document.save(docx_path)

def convert_directory_pdf_to_docx(pdf_directory, docx_directory):
    # Create the output directory if it doesn't exist
    if not os.path.exists(docx_directory):
        os.makedirs(docx_directory)

    # Get a list of PDF files in the input directory
    pdf_files = glob.glob(os.path.join(pdf_directory, '*.pdf'))

    total_files = len(pdf_files)
    print(f"Converting {total_files} PDF files to DOCX...")

    for idx, pdf_file in enumerate(pdf_files, 1):
        # Construct the output file path
        base_name = os.path.basename(pdf_file)
        file_name = os.path.splitext(base_name)[0]
        docx_file = os.path.join(docx_directory, f'{file_name}.docx')

        # Convert the PDF file to DOCX
        pdf_to_docx(pdf_file, docx_file)

        # Print the progress
        print(f"Processed file {idx}/{total_files}")

    print("Conversion completed successfully!")

# Example usage
pdf_directory = r'C:\Users\RADHA\Documents\gentask\samplepdf'
docx_directory = r'C:\Users\RADHA\Documents\gentask\samplepdf2docx'

convert_directory_pdf_to_docx(pdf_directory, docx_directory)


Converting 1 PDF files to DOCX...
Processed file 1/1
Conversion completed successfully!


In [14]:
import os
import glob
import docx
import spacy
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
from PyPDF2 import PdfFileReader, PdfFileWriter

# Load the custom NER model
nlp = spacy.load("./model-last")

def extract_custom_ner_entities(text):
    """
    Extract custom NER entities from text
    """
    doc = nlp(text)
    custom_ner_entities = []
    for ent in doc.ents:
        custom_ner_entities.append((ent.label_, ent.text))
    return custom_ner_entities

def redact_matching_data(paragraph, matched_entities):
    """
    Redact matching entities in a paragraph
    """
    for entity_label, entity_text in matched_entities:
        for run in paragraph.runs:
            if entity_text in run.text:
                run_text = run.text.replace(entity_text, "[REDACTED]")
                run.clear()  # Remove the existing run
                new_run = paragraph.add_run(run_text)  # Create a new run with the redacted text
                font = new_run.font
                font.highlight_color = docx.enum.text.WD_COLOR_INDEX.BLACK  # Apply black highlight color for redacted text

def redact_docx_file(input_file, output_file):
    """
    Redact custom NER entities in a DOCX file and save the redacted file
    """
    doc = docx.Document(input_file)

    for paragraph in doc.paragraphs:
        text = paragraph.text
        matched_entities = extract_custom_ner_entities(text)
        redact_matching_data(paragraph, matched_entities)

    doc.save(output_file)

def redact_pdf_file(input_file, output_file):
    """
    Redact custom NER entities in a PDF file and save the redacted file
    """
    pdf_reader = PdfFileReader(input_file)
    pdf_writer = PdfFileWriter()

    for page_num in range(pdf_reader.numPages):
        page = pdf_reader.getPage(page_num)
        text = page.extract_text()
        matched_entities = extract_custom_ner_entities(text)
        redacted_text = text

        for entity_label, entity_text in matched_entities:
            redacted_text = redacted_text.replace(entity_text, "[REDACTED]")

        page = page.extract_text().replace(text, redacted_text)
        pdf_writer.addPage(page)

    with open(output_file, 'wb') as output_pdf:
        pdf_writer.write(output_pdf)

def redact_files_in_directory(input_directory, output_directory):
    """
    Redact custom NER entities in all PDF and DOCX files in the input directory and save the redacted files in the output directory
    """
    os.makedirs(output_directory, exist_ok=True)
    files = glob.glob(os.path.join(input_directory, "*.*"))

    for file in files:
        file_ext = os.path.splitext(file)[1].lower()
        file_name = os.path.basename(file)
        output_file = os.path.join(output_directory, file_name)

        if file_ext == ".docx":
            redact_docx_file(file, output_file)
            print(f"Redacted DOCX file saved: {output_file}")
        elif file_ext == ".pdf":
            redact_pdf_file(file, output_file)
            print(f"Redacted PDF file saved: {output_file}")

# Specify the input and output directories
input_directory = '/home/ec2-user/SageMaker/synthetic_resumes3/'
output_directory = '/home/ec2-user/SageMaker/syn_redact/'

# Redact custom NER entities in all PDF and DOCX files in the input directory and save the redacted files in the output directory
redact_files_in_directory(input_directory, output_directory)


ModuleNotFoundError: No module named 'spacy'

In [ ]:
import os
import glob
import docx
import spacy
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
from PyPDF4 import PdfFileReader, PdfFileWriter

# Load the custom NER model
nlp = spacy.load("./model-last")

def extract_custom_ner_entities(text):
    """
    Extract custom NER entities from text
    """
    doc = nlp(text)
    custom_ner_entities = []
    for ent in doc.ents:
        custom_ner_entities.append((ent.label_, ent.text))
    return custom_ner_entities

def redact_matching_data(paragraph, matched_entities):
    """
    Redact matching entities in a paragraph
    """
    for entity_label, entity_text in matched_entities:
        for run in paragraph.runs:
            if entity_text in run.text:
                run_text = run.text.replace(entity_text, "[REDACTED]")
                run.clear()  # Remove the existing run
                new_run = paragraph.add_run(run_text)  # Create a new run with the redacted text
                font = new_run.font
                font.highlight_color = docx.enum.text.WD_COLOR_INDEX.BLACK  # Apply black highlight color for redacted text

def redact_docx_file(input_file, output_file):
    """
    Redact custom NER entities in a DOCX file and save the redacted file
    """
    doc = docx.Document(input_file)

    for paragraph in doc.paragraphs:
        text = paragraph.text
        matched_entities = extract_custom_ner_entities(text)
        redact_matching_data(paragraph, matched_entities)

    doc.save(output_file)

def redact_pdf_file(input_file, output_file):
    """
    Redact custom NER entities in a PDF file and save the redacted file
    """
    pdf_reader = PdfFileReader(input_file)
    pdf_writer = PdfFileWriter()

    for page_num in range(pdf_reader.numPages):
        page = pdf_reader.getPage(page_num)
        text = page.extractText()
        matched_entities = extract_custom_ner_entities(text)
        redacted_text = text

        for entity_label, entity_text in matched_entities:
            redacted_text = redacted_text.replace(entity_text, "[REDACTED]")

        page = page.extractText().replace(text, redacted_text)
        pdf_writer.addPage(page)

    with open(output_file, 'wb') as output_pdf:
        pdf_writer.write(output_pdf)

def redact_files_in_directory(input_directory, output_directory):
    """
    Redact custom NER entities in all PDF and DOCX files in the input directory and save the redacted files in the output directory
    """
    os.makedirs(output_directory, exist_ok=True)
    files = glob.glob(os.path.join(input_directory, "*.*"))

    for file in files:
        file_ext = os.path.splitext(file)[1].lower()
        file_name = os.path.basename(file)
        output_file = os.path.join(output_directory, file_name)

        if file_ext == ".docx":
            redact_docx_file(file, output_file)
            print(f"Redacted DOCX file saved: {output_file}")
        elif file_ext == ".pdf":
            redact_pdf_file(file, output_file)
            print(f"Redacted PDF file saved: {output_file}")

# Specify the input and output directories
input_directory = '/home/ec2-user/SageMaker/synthetic_resumes3/'
output_directory = '/home/ec2-user/SageMaker/syn_redact/'

# Redact custom NER entities in all PDF and DOCX files in the input directory and save the redacted files in the output directory
redact_files_in_directory(input_directory, output_directory)
